In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import notebook_login

notebook_login()

In [5]:

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    logging,
    set_seed,
    BitsAndBytesConfig,
)
SEED = 0
set_seed(SEED)

In [7]:
from peft import PeftModel
import torch

MODEL = "bigcode/starcoderbase-1b"
# load the original model first
tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    quantization_config=None,
    device_map=None,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
).cuda()

# merge fine-tuned weights with the base model
peft_model_id = f"Warkawik/code_example"
model = PeftModel.from_pretrained(base_model, peft_model_id)
model.merge_and_unload()

adapter_config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/22.2M [00:00<?, ?B/s]

GPTBigCodeForCausalLM(
  (transformer): GPTBigCodeModel(
    (wte): Embedding(49152, 2048)
    (wpe): Embedding(8192, 2048)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPTBigCodeBlock(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPTBigCodeAttention(
          (c_attn): Linear(in_features=2048, out_features=2304, bias=True)
          (c_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTBigCodeMLP(
          (c_fc): Linear(in_features=2048, out_features=8192, bias=True)
          (c_proj): Linear(in_features=8192, out_features=2048, bias=True)
          (act): PytorchGELUTanh()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e

In [8]:
def get_code_completion(prefix, suffix):
    text = prompt = f"""<fim_prefix>{prefix}<fim_suffix>{suffix}<fim_middle>"""
    model.eval()
    outputs = model.generate(
        input_ids=tokenizer(text, return_tensors="pt").input_ids.cuda(),
        max_new_tokens=128,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.0,
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

In [11]:
prefix = """import cmath

#create a reverse shell tcp for linux
"""
suffix =""""""

print(get_code_completion(prefix, suffix))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


import cmath

#create a reverse shell tcp for linux

from pyshark import LiveStream

#create a tcp stream

stream = LiveStream(
    interface='eth0',
    filter='not (tcp.dst_port == 22 or tcp.dst_port == 23 or tcp.dst_port == 25565 or tcp.dst_port == 25566 or tcp.dst_port == 25567 or tcp.dst_port == 25568 or tcp.dst_port == 25569 or tcp.dst_port == 2557
